In [7]:
# Импорт необходимых библиотек
import pandas as pd
import json
import xml.etree.ElementTree as ET
from lxml import etree as lxml_etree
import unittest

from csv_to_json import *
from xml_parser import *



In [9]:
# Функция для преобразования CSV в JSON
def csv_to_json(csv_file, delimiter=','):
    df = pd.read_csv(csv_file, delimiter=delimiter)
    json_data = df.to_dict(orient='records')
    return json_data

# Пример использования csv_to_json
csv_file = 'test_data/example.csv'
data = csv_to_json(csv_file)
print(data)


[{'Name': 'Alice', 'Age': 30, 'City': 'New York', 'Phone': '123-456-7890', 'Email': 'alice@example.com', 'DateOfBirth': '1993-01-15'}, {'Name': 'Bob', 'Age': 25, 'City': 'Los Angeles', 'Phone': '987-654-3210', 'Email': 'bob@example.com', 'DateOfBirth': '1998-05-23'}, {'Name': 'Charlie', 'Age': 35, 'City': 'Chicago', 'Phone': '555-555-5555', 'Email': 'u9G5h@example.com', 'DateOfBirth': '1988-10-10'}]


In [14]:
# Функция для сохранения JSON в файл
def save_json(data, output_file, individual_files=False):
    if individual_files:
        for i, item in enumerate(data):
            with open(f'{output_file}_{i}.json', 'w') as f:
                json.dump(item, f, indent=4)
    else:
        with open(output_file, 'w') as f:
            json.dump(data, f, indent=4)

# Пример использования save_json
json_file = 'output_jsons/output.json'
save_json(data, json_file, individual_files=False)


In [17]:
# Функция для сохранения JSON в файл
def save_json(data, output_file, individual_files=False):
    if individual_files:
        for i, item in enumerate(data):
            with open(f'{output_file}_{i}.json', 'w') as f:
                json.dump(item, f, indent=4)
    else:
        with open(output_file, 'w') as f:
            json.dump(data, f, indent=4)

# Пример использования save_json
json_file = 'output_jsons/output.json'
save_json(data, json_file, individual_files=False)


In [18]:
# Функция для создания XML элемента
def create_xml_element(parent, tag, text=None, attributes=None):
    if attributes is None:
        attributes = {}
    element = ET.SubElement(parent, tag, attrib=attributes)
    if text:
        element.text = str(text)
    return element

# Рекурсивная функция для генерации XML
def _generate_xml_recursive(parent, data):
    if isinstance(data, dict):
        for key, value in data.items():
            child = create_xml_element(parent, key)
            _generate_xml_recursive(child, value)
    elif isinstance(data, list):
        for item in data:
            child = create_xml_element(parent, 'item')
            _generate_xml_recursive(child, item)
    else:
        parent.text = str(data)

# Функция для генерации XML из данных
def generate_xml(data, root_tag='root'):
    root = ET.Element(root_tag)
    _generate_xml_recursive(root, data)
    return ET.tostring(root, encoding='unicode')

# Пример использования generate_xml
root_tag = 'people'
xml_string = generate_xml(data, root_tag)
print(xml_string)


<people><item><Name>Alice</Name><Age>30</Age><City>New York</City><Phone>123-456-7890</Phone><Email>alice@example.com</Email><DateOfBirth>1993-01-15</DateOfBirth></item><item><Name>Bob</Name><Age>25</Age><City>Los Angeles</City><Phone>987-654-3210</Phone><Email>bob@example.com</Email><DateOfBirth>1998-05-23</DateOfBirth></item><item><Name>Charlie</Name><Age>35</Age><City>Chicago</City><Phone>555-555-5555</Phone><Email>u9G5h@example.com</Email><DateOfBirth>1988-10-10</DateOfBirth></item></people>


In [21]:
# Функция для сохранения XML в файл
def save_xml(xml_string, output_file):
    with open(output_file, 'w') as f:
        f.write(xml_string)

# Пример использования save_xml
xml_file = 'xml_parser/out_xml/generated_xml.xml'
save_xml(xml_string, xml_file)


In [22]:
# Функция для проверки XML по XSD схеме
def validate_xml(xml_string, xsd_file):
    xml_doc = lxml_etree.fromstring(xml_string)
    schema = lxml_etree.XMLSchema(lxml_etree.parse(xsd_file))
    return schema.validate(xml_doc)

# Пример использования validate_xml
xsd_file = 'xml_parser/test_xml/schema.xsd'
is_valid = validate_xml(xml_string, xsd_file)
print(f'XML is valid: {is_valid}')


XML is valid: False


In [23]:
# Функция для чтения XML из файла
def read_xml_from_file(xml_file):
    with open(xml_file, 'r') as f:
        return f.read()

# Пример использования read_xml_from_file
xml_content = read_xml_from_file(xml_file)
print(xml_content)


<people><item><Name>Alice</Name><Age>30</Age><City>New York</City><Phone>123-456-7890</Phone><Email>alice@example.com</Email><DateOfBirth>1993-01-15</DateOfBirth></item><item><Name>Bob</Name><Age>25</Age><City>Los Angeles</City><Phone>987-654-3210</Phone><Email>bob@example.com</Email><DateOfBirth>1998-05-23</DateOfBirth></item><item><Name>Charlie</Name><Age>35</Age><City>Chicago</City><Phone>555-555-5555</Phone><Email>u9G5h@example.com</Email><DateOfBirth>1988-10-10</DateOfBirth></item></people>


In [24]:
# Определение класса для юнит-тестов JSON утилит
class TestJSONUtils(unittest.TestCase):
    def test_save_json(self):
        data = csv_to_json(csv_file)
        save_json(data, json_file, individual_files=False)
        with open(json_file, 'r') as f:
            loaded_data = json.load(f)
        self.assertEqual(data, loaded_data)

# Определение класса для юнит-тестов XML утилит
class TestXMLUtils(unittest.TestCase):
    def test_generate_xml(self):
        data = csv_to_json(csv_file)
        xml_string = generate_xml(data, root_tag)
        self.assertIn('<people>', xml_string)
        self.assertIn('</people>', xml_string)

    def test_validate_xml_valid(self):
        data = csv_to_json(csv_file)
        xml_string = generate_xml(data, root_tag)
        is_valid = validate_xml(xml_string, xsd_file)
        self.assertTrue(is_valid)

    def test_validate_xml_invalid(self):
        invalid_xml = '<people><person><name>Alice</name><age>30</age><city>New York</city></person></people>'
        is_valid = validate_xml(invalid_xml, xsd_file)
        self.assertFalse(is_valid)

    def test_read_xml_from_file(self):
        xml_content = read_xml_from_file(xml_file)
        self.assertIn('<people>', xml_content)
        self.assertIn('</people>', xml_content)

    def test_write_xml_to_file(self):
        data = csv_to_json(csv_file)
        xml_string = generate_xml(data, root_tag)
        write_xml_to_file(xml_string, xml_file)
        with open(xml_file, 'r') as f:
            loaded_xml = f.read()
        self.assertEqual(xml_string, loaded_xml)

# Запуск юнит-тестов
unittest.main(argv=[''], exit=False)


...FFE
ERROR: test_write_xml_to_file (__main__.TestXMLUtils.test_write_xml_to_file)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_4176\2511754388.py", line 37, in test_write_xml_to_file
    write_xml_to_file(xml_string, xml_file)
    ^^^^^^^^^^^^^^^^^
NameError: name 'write_xml_to_file' is not defined

FAIL: test_validate_xml_invalid (__main__.TestXMLUtils.test_validate_xml_invalid)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_4176\2511754388.py", line 27, in test_validate_xml_invalid
    self.assertFalse(is_valid)
AssertionError: True is not false

FAIL: test_validate_xml_valid (__main__.TestXMLUtils.test_validate_xml_valid)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\T